# 単語のポジネガによるテキストランキング

## 参考

http://www.statsbeginner.net/entry/2017/05/07/091435  
http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html

In [3]:
# -*- coding: utf-8 -*-
import re
import csv
import time
import pandas as pd
import matplotlib.pyplot as plt
import MeCab
mecab = MeCab.Tagger(' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse("")  # 追加
import random
from statistics import mean, median,variance,stdev
import numpy as np

In [4]:
# PN Tableを読み込み
# パスは各自適当なものになります
pn_df = pd.read_csv('PN_dict/pn_ja.txt',\
                    sep=':',
                    encoding='utf-8',
                    names=('Word','Reading','POS', 'PN')
                   )

# PN Tableをデータフレームからdict型に変換しておく
word_list = list(pn_df['Word'])
pn_list = list(pn_df['PN'])  # 中身の型はnumpy.float64
pn_dict = dict(zip(word_list, pn_list))

In [5]:
pn_df

,Word,Reading,POS,PN
0,優れる,すぐれる,動詞,1.000000
1,良い,よい,形容詞,0.999995
2,喜ぶ,よろこぶ,動詞,0.999979
3,褒める,ほめる,動詞,0.999979
4,めでたい,めでたい,形容詞,0.999645
5,賢い,かしこい,形容詞,0.999486
6,善い,いい,形容詞,0.999314
7,適す,てきす,動詞,0.999295
8,天晴,あっぱれ,名詞,0.999267
9,祝う,いわう,動詞,0.999122


In [6]:
def get_diclist(text):
    if '、' in text:
        text = text.replace('、', '')
    
    parsed = mecab.parse(text)      # 形態素解析結果（改行を含む文字列として得られる）
    lines = parsed.split('\n')  # 解析結果を1行（1語）ごとに分けてリストにする
    lines = lines[0:-2]         # 後ろ2行は不要なので削除
    diclist = []
    for word in lines:
        l = re.split('\t|,',word)  # 各行はタブとカンマで区切られてるので
        if not l[1] == '助詞' and not l[1] == '記号' and not l[1] == '助動詞' and not l[7] == '*':
            d = {'Surface':l[0], 'POS1':l[1], 'POS2':l[2], 'BaseForm':l[7]}
            diclist.append(d)
    return(diclist)

In [7]:
# 形態素解析結果の単語ごとdictデータにPN値を追加する関数
def add_pnvalue(diclist_old):
    diclist_new = []
    for word in diclist_old:
        base = word['BaseForm']        # 個々の辞書から基本形を取得
        if base in pn_dict:
            pn = float(pn_dict[base])  # 中身の型があれなので
        else:
            pn = 'notfound'            # その語がPN Tableになかった場合
        word['PN'] = pn
        diclist_new.append(word)
    return(diclist_new)

In [8]:
# 各ツイートのPN平均値をとる関数
def get_pnmean(diclist):
    pn_list = []
    for word in diclist:
        pn = word['PN']
        if pn != 'notfound':
            pn_list.append(pn)  # notfoundだった場合は追加もしない            
    if len(pn_list) > 0:        # 「全部notfound」じゃなければ
        pnmean = mean(pn_list)
    else:
        pnmean = 0              # 全部notfoundならゼロにする
    return(pnmean)

In [9]:
# 各ツイートのPN平均値をとる関数
def get_pnaverage(diclist):
    pn_list = []
    for word in diclist:
        pn = word['PN']
        if pn != 'notfound':
            pn_list.append(pn)  # notfoundだった場合は追加もしない            
    if len(pn_list) > 0:        # 「全部notfound」じゃなければ
        pnmean = np.average(pn_list)
    else:
        pnmean = 0              # 全部notfoundならゼロにする
    return(pnmean)

In [10]:
text_test = '昨日はラーメンを食べました．美味しかった．'

In [11]:
text_test

'昨日はラーメンを食べました．美味しかった．'

テキストデータから **「助詞」「助動詞」「記号」以外の単語（日本語）** に対して形態素解析

In [12]:
get_diclist(text_test)

[{'BaseForm': '昨日', 'POS1': '名詞', 'POS2': '副詞可能', 'Surface': '昨日'},
 {'BaseForm': 'ラーメン', 'POS1': '名詞', 'POS2': '一般', 'Surface': 'ラーメン'},
 {'BaseForm': '食べる', 'POS1': '動詞', 'POS2': '自立', 'Surface': '食べ'},
 {'BaseForm': '美味しい', 'POS1': '形容詞', 'POS2': '自立', 'Surface': '美味しかっ'}]

各単語の **PN値** を取得する

In [13]:
dl_test = get_diclist(text_test)

In [14]:
add_pnvalue(dl_test)

[{'BaseForm': '昨日',
  'PN': -0.368417,
  'POS1': '名詞',
  'POS2': '副詞可能',
  'Surface': '昨日'},
 {'BaseForm': 'ラーメン',
  'PN': -0.293937,
  'POS1': '名詞',
  'POS2': '一般',
  'Surface': 'ラーメン'},
 {'BaseForm': '食べる',
  'PN': -0.9683370000000001,
  'POS1': '動詞',
  'POS2': '自立',
  'Surface': '食べ'},
 {'BaseForm': '美味しい',
  'PN': 0.99136,
  'POS1': '形容詞',
  'POS2': '自立',
  'Surface': '美味しかっ'}]

各単語の **PN値の平均** をとってテキストのスコアとする

In [15]:
dl_test = add_pnvalue(dl_test)

In [16]:
get_pnmean(dl_test)

-0.15983275000000002

In [17]:
get_pnaverage(dl_test)

-0.15983275000000002

## スコアによってテキストをランク付けする

In [18]:
text_list = [
    '昨日はラーメンを食べました．美味しかった．',
    '明日は晴れるといいな',
    '10日放送の「中居正広のミになる図書館」（テレビ朝日系）で、SMAPの中居正広が、篠原信一の過去の勘違いを明かす一幕があった'
]

In [19]:
text_list

['昨日はラーメンを食べました．美味しかった．',
 '明日は晴れるといいな',
 '10日放送の「中居正広のミになる図書館」（テレビ朝日系）で、SMAPの中居正広が、篠原信一の過去の勘違いを明かす一幕があった']

In [20]:
pn_text_dict = {}

for text in text_list:
    dl_test = get_diclist(text)
    dl_test = add_pnvalue(dl_test)
    score = get_pnmean(dl_test)
    
    pn_text_dict[text] = score
    
pn_text_dict = sorted(pn_text_dict.items(), key=lambda x: -x[1])

for k, v in pn_text_dict:
    print(str(k))
    print(str(v))
    print()

明日は晴れるといいな
0.19169049999999999

昨日はラーメンを食べました．美味しかった．
-0.15983275000000002

10日放送の「中居正広のミになる図書館」（テレビ朝日系）で、SMAPの中居正広が、篠原信一の過去の勘違いを明かす一幕があった
-0.561006



# 問題点

* 辞書に大きく依存する（今回使用したのは自動生成された辞書）
* 適切な文を断定することはできない（あくまでランク付けした結果を提示）